In [594]:
import pandas as pd
import numpy as np
import re

In [509]:
traintanic = pd.read_csv('train.csv')
testanic = pd.read_csv('test.csv')

Features:
Ticket class, sex, age, parent-child, sibling-spouse, Tune: nationality (split off last names and classify non-Anglos by name ending)

Start with decision tree, then do KNN, then SVM
See what happens when combined

To do: separate out deck from cabin number

Assign deck as well as fore, amidships or aft cabin numbers based on ticket class

Check on how exactly the ship sank - 

Create column for young children (try different limit downwards from 16)

Ignore ticket & fare

Ignore port of embarkation


'Foreign' name endings:
off
ov
cher
sky
ski
ig
nen
sen
ik
strom
ic
sen
blom
lund
og

In [512]:
traintanic['Last_Name'] = traintanic['Name'].str.split(',', n = 1, expand = True)

In [643]:
traintanic['Is Non-Anglo Western?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

testanic['Is Non-Anglo Western?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

In [644]:
traintanic['Is Eastern?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

testanic['Is Eastern?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

In [518]:
traintanic['Is 1st Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

testanic['Is 1st Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

In [519]:
traintanic['Is 3rd Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

testanic['Is 3rd Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

In [520]:
traintanic['Deck'] = traintanic.Cabin.str.get(0)
traintanic['Cabin'] = traintanic.Cabin.astype('object')

testanic['Deck'] = testanic.Cabin.str.get(0)
testanic['Cabin'] = testanic.Cabin.astype('object')

In [521]:
traintanic['Cabin_No'] = traintanic['Cabin'].str[1:4]

testanic['Cabin_No'] = testanic['Cabin'].str[1:4]

In [522]:
traintanic.Deck = traintanic.Deck.fillna(0)
traintanic.Cabin_No = traintanic.Cabin_No.fillna(0)

testanic.Deck = testanic.Deck.fillna(0)
testanic.Cabin_No = testanic.Cabin_No.fillna(0)

In [616]:
traintanic['Is Child?'] = traintanic.Age.apply(lambda x: 1 if x <= 12 else 0)
traintanic['Is Young Adult?'] = traintanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

testanic['Is Child?'] = testanic.Age.apply(lambda x: 1 if x <= 12 else 0)
testanic['Is Young Adult?'] = testanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

In [524]:
traintanic['Is Not Young?'] = traintanic.Age.apply(lambda x: 1 if x > 40 else 0)

testanic['Is Not Young?'] = testanic.Age.apply(lambda x: 1 if x > 40 else 0)

In [525]:
traintanic['Big Family?'] = traintanic.Parch.apply(lambda x: 1 if x > 2 else 0)

testanic['Big Family?'] = testanic.Parch.apply(lambda x: 1 if x > 2 else 0)

In [619]:
traintanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

testanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

C:\Users\teren\miniconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [528]:
traintanic.Cabin_No.replace(to_replace=' G7', value=7, inplace=True)
traintanic.Cabin_No.replace(to_replace=' G6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace=' E6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace='', value=0, inplace=True)

In [667]:
testanic.Cabin_No.replace(to_replace=' [A-Z]', value='', inplace=True, regex=True)
testanic.Cabin_No.replace(to_replace='', value='0', inplace=True, regex=True)

In [529]:
traintanic['Cabin_No'] = traintanic['Cabin_No'].astype('int64')

testanic['Cabin_No'] = testanic['Cabin_No'].astype('int64')

In [530]:
traintanic['Cabin Fore?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

testanic['Cabin Fore?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

In [531]:
traintanic['Cabin Aft?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

testanic['Cabin Aft?'] = testanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

In [532]:
traintanic['Cabin Amidships?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

testanic['Cabin Amidships?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

In [533]:
traintanic = traintanic.drop(columns=['PassengerId','Ticket','Fare'])

testanic = testanic.drop(columns=['PassengerId','Ticket','Fare'])

In [534]:
traintanic['Sex-int'] = traintanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

testanic['Sex-int'] = testanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

In [535]:
traintanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Survived               891 non-null    int64  
 1   Pclass                 891 non-null    int64  
 2   Name                   891 non-null    object 
 3   Sex                    891 non-null    object 
 4   Age                    714 non-null    float64
 5   SibSp                  891 non-null    int64  
 6   Parch                  891 non-null    int64  
 7   Embarked               889 non-null    object 
 8   Last_Name              891 non-null    object 
 9   Is Non-Anglo Western?  891 non-null    int64  
 10  Is Eastern?            891 non-null    int64  
 11  Is Non-Anglo?          891 non-null    int64  
 12  Is 1st Class?          891 non-null    int64  
 13  Is 3rd Class?          891 non-null    int64  
 14  Deck                   891 non-null    object 
 15  Cabin_

In [536]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [676]:
forest = RandomForestClassifier(n_estimators = 10000, random_state = 1)
traintanic_data, valitanic_data, traintanic_labels, valitanic_labels = train_test_split(traintanic[[\
'Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Is Child?','Is Young Adult?','Is Not Young?',\
            'Cabin Fore?','Cabin Aft?','Cabin Amidships?']],\
           traintanic['Survived'], test_size = 0.2)
forest.fit(traintanic_data, traintanic_labels)

print(forest.score(valitanic_data, valitanic_labels))

0.8044692737430168


In [678]:
testanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Pclass                 418 non-null    int64  
 1   Name                   418 non-null    object 
 2   Sex                    418 non-null    object 
 3   Age                    332 non-null    float64
 4   SibSp                  418 non-null    int64  
 5   Parch                  418 non-null    int64  
 6   Cabin                  91 non-null     object 
 7   Embarked               418 non-null    object 
 8   Last_Name              418 non-null    object 
 9   Is Non-Anglo Western?  418 non-null    int64  
 10  Is Eastern?            418 non-null    int64  
 11  Is 1st Class?          418 non-null    int64  
 12  Is 3rd Class?          418 non-null    int64  
 13  Deck                   418 non-null    object 
 14  Cabin_No               418 non-null    int64  
 15  Is Chi

In [679]:
testanic_data = testanic[[\
'Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Is Child?','Is Young Adult?','Is Not Young?',\
            'Cabin Fore?','Cabin Aft?','Cabin Amidships?']]

In [680]:
testanic_predict = forest.predict(testanic_data)

In [682]:
testanic['Survived'] = testanic_predict

In [489]:
feature_importances = list(forest.feature_importances_)
features_list = ['Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch','Is Non-Anglo?','Is Young Boy?',\
            'Is Young Woman?','Is Not Young?','Cabin Fore?','Cabin Aft?','Cabin Amidships?']

In [490]:
for feature, value in zip(features_list, feature_importances):
    print(f'{feature}: {value}')

Is 1st Class?: 0.07881739848545546
Is 3rd Class?: 0.12411116075748313
Sex-int: 0.34243405408660327
SibSp: 0.12654300299969265
Parch: 0.08920871875653362
Is Non-Anglo?: 0.039159020279354605
Is Young Boy?: 0.043227565473484976
Is Young Woman?: 0.0957064993448946
Is Not Young?: 0.028231715229254536
Cabin Fore?: 0.014147980353781248
Cabin Aft?: 0.010918336605626425
Cabin Amidships?: 0.007494547627835345
